## Author: Etienne Bonnassieux
## Comments, questions? Contact etienne.bonnassieux@uni-wuerzburg.de

# **I. Prerequisites**

In this jupyter notebook, we will learn how to install & run DDFacet on a LOFAR dataset. For this, you must first have singularity installed. You can create your own singularity image from the DDF pipeline repository, found here:

https://github.com/mhardcastle/ddf-pipeline/tree/master

It will require the following command:

In [ ]:
singularity build --fakeroot $PWD/ddf.sif $PWD/ddf-py3.singularity

You can probably go for a coffee break while it builds. For the purposes of this tutorial, you should already have downloaded this output before the day, and can proceed by setting up your environment. Make sure you add all relevant directories that you need to access to the following:

In [ ]:
singularity shell -B$PWD -B/home/$USER -B/data -B/scratch ~/DDFSingularity/ddf_backup.sif

# **II. Introduction**


The aim of this tutorial is to calibrate & image a small LOFAR dataset. Here is our order of operations:
- Have a dataset (if using my provided one, will need ~1 GB)
- Create an initial image from our dataset
- Create a catalogue of our field to set up the direction-dependent calibration
- Inspect the image and set up the direction-dependent calibration
- Run the direction-dependent calibration
- Perform direction-dependent imaging

A test dataset for this tutorial will have been shared ahead of time. Similarly, a Singularity image, allowing you to conveniently use a Linux environment adapted for this tutorial, will have been shared. More perennial links will eventually be added here. TODO: do that + add description and so on.

In [ ]:
commands:

time /usr/local/src/DDFacet/DDFacet/DDF.py --Data-MS M31.143MHz.2h.MS --Data-ColName DATA --Output-Name IMAGES/test.m31 --Output-Mode Clean --Output-Also all --Image-NPix 5000 --Image-Cell 5 --Facets-NFacets 11 --Weight-Mode Briggs --Weight-Robust 0 --RIME-DecorrMode FT --Parallel-NCPU 64 --Beam-Model None --Deconv-Mode HMP --Deconv-MaxMajorIter 20 --Debug-Pdb never --Cache-Reset 1 --Deconv-AllowNegative 0 --Deconv-Gain 0.4 --Deconv-MaxMinorIter 2000
MakeModel.py --BaseImageName IMAGES/test.m31 --NCluster 15
time /usr/local/src/killMS/killMS/kMS.py --MSName M31.143MHz.2h.MS --InCol DATA --SkyModel IMAGES/test.m31.npy --NCPU 64 --dt 10 --NChanSols 4 --DebugPdb 0 --OutSolsName ddcal.pass0 --SolsDir DDcal
time /usr/local/src/DDFacet/DDFacet/DDF.py --Data-MS M31.143MHz.2h.MS --Data-ColName DATA --Output-Name IMAGES/test.m31.ddcal0 --Output-Mode Clean --Output-Also all --Image-NPix 5000 --Image-Cell 5 --Facets-NFacets 11 --Weight-Mode Briggs --Weight-Robust 0 --RIME-DecorrMode FT --Parallel-NCPU 64 --Beam-Model None --Deconv-Mode HMP --Deconv-MaxMajorIter 20 --Debug-Pdb never --Cache-Reset 0 --Deconv-AllowNegative 0 --Deconv-Gain 0.4 --Deconv-MaxMinorIter 2000 --DDESolutions-DDSols ddcal.pass0 --DDESolutions-DDModeGrid P --DDESolutions-DDModeDeGrid P --DDESolutions-SolsDir ./DDcal/ --DDESolutions-JonesMode Diag



# **III. Making the first DDF image of the field **


For now, we will assume that you are starting with only visibility data, rather than a pybdsf gaul catalogue. Thus, it is useful to image your field in order to see what you're working with and how many tessels you may want to create. The options for DDFacet can be invoked by using the --help or -h command from within the Singularity, as below:

In [1]:
DDF.py -h

NameError: name 'DDF' is not defined

As you can see, there's quite a bit!...for you, the vast majority of them can be safely ignored. For starters, let's make a small image with our dataset. Since I'm running this on my laptop, I don't want to use too many processors, nor make too large an image (the defaults for NCPU and NPix are 10 and 5000 respectively).

In [ ]:
time /usr/local/src/DDFacet/DDFacet/DDF.py --Data-MS M31.143MHz.2h.MS --Data-ColName DATA --Output-Name IMAGES/test.m31 --Output-Mode Clean --Output-Also all --Image-NPix 5000 --Image-Cell 5 --Facets-NFacets 11 --Weight-Mode Briggs --Weight-Robust 0 --RIME-DecorrMode FT --Parallel-NCPU 64 --Beam-Model None --Deconv-Mode HMP --Deconv-MaxMajorIter 20 --Debug-Pdb never --Cache-Reset 1 --Deconv-AllowNegative 0 --Deconv-Gain 0.4 --Deconv-MaxMinorIter 2000

If all went well, you should now have a restored image of our 2h of data. Note that the run will have generated a parset file. Although we do not recommend this, it is possible to script or automate DDF run by invoking parsets adn changing only some parameters, as follows:

In [ ]:
DDF.py whatever.parset --Output-Mode PSF --Image-NPix 2 ...

Let's look at the contents of our latest parset output, image.parset:

In [ ]:
[Data]
MS = DATA/L242400_SB095_uv.dppp.4h-4.5h.MS 
ColName = CORRECTED_DATA 
ChunkHours = 0.0 
Sort = False 

[Predict]
ColName = None 
MaskSquare = None 
FromImage = None 
InitDicoModel = None 
Overwrite = True 

[Selection]
Field = 0 
DDID = 0 
TaQL =  
ChanStart = 0 
ChanEnd = -1 
ChanStep = 1 
FlagAnts =  
UVRangeKm = [0, 2000] 
TimeRange =  
DistMaxToCore =  

[Output]
Mode = Clean 
Name = image 
ShiftFacetsFile = None 
RestoringBeam = None 
Also =  
Cubes =  
Images = DdPAMRIikz 
alphathreshold = 7 
alphamaskthreshold = 15 
StokesResidues = I 

[Image]
NPix = 500 
Cell = 5.0 
PhaseCenterRADEC = align 
SidelobeSearchWindow = 200 

[Facets]
NFacets = 3 
CatNodes = None 
DiamMax = 180.0 
DiamMin = 0.0 
PSFOversize = 1.0 
PSFFacets = 0 
Padding = 1.7 
Circumcision = 0 

[Weight]
ColName = WEIGHT_SPECTRUM 
Mode = Briggs 
MFS = True 
Robust = 0.0 
SuperUniform = 1.0 

[RIME]
Precision = S 
PolMode = I 
FFTMachine = FFTW 
ForwardMode = BDA-degrid 
BackwardMode = BDA-grid 
DecorrMode =  
DecorrLocation = Edge 

[CF]
OverS = 11 
Support = 7 
Nw = 100 
wmax = 0.0 

[Comp]
GridDecorr = 0.02 
GridFoV = Facet 
DegridDecorr = 0.02 
DegridFoV = Facet 
Sparsification = 0 
BDAMode = 1 
BDAJones = 0 

[Parallel]
NCPU = 4 
Affinity = 1 
MainProcessAffinity = 0 

[Cache]
Reset = True 
SmoothBeam = auto 
Weight = auto 
PSF = auto 
Dirty = auto 
VisData = auto 
LastResidual = True 
Dir =  
DirWisdomFFTW = ~/.fftw_wisdom 
ResetWisdom = False 
CF = True 
HMP = False 

[Beam]
Model = None 
At = facet 
LOFARBeamMode = AE 
NBand = 0 
CenterNorm = False 
Smooth = False 
SmoothNPix = 11 
FITSFile = beam_$(corr)_$(reim).fits 
FITSFeed = None 
DtBeamMin = 5.0 
FITSParAngleIncDeg = 5.0 
FITSLAxis = -X 
FITSMAxis = Y 
FITSVerbosity = 0 

[Freq]
BandMHz = 0.0 
DegridBandMHz = 0.0 
NBand = 1 
NDegridBand = 0 

[DDESolutions]
DDSols =  
SolsDir = None 
GlobalNorm = None 
JonesNormList = AP 
JonesMode = Full 
DDModeGrid = AP 
DDModeDeGrid = AP 
ScaleAmpGrid = 0 
ScaleAmpDeGrid = 0 
CalibErr = 10.0 
Type = Nearest 
Scale = 1.0 
gamma = 4.0 
RestoreSub = False 
ReWeightSNR = 0.0 

[Deconv]
Mode = HMP 
MaxMajorIter = 20 
MaxMinorIter = 20000 
AllowNegative = True 
Gain = 0.1 
FluxThreshold = 0.0 
CycleFactor = 0.0 
RMSFactor = 0.0 
PeakFactor = 0.15 
PrevPeakFactor = 0.0 
NumRMSSamples = 10000 
ApproximatePSF = 0 
PSFBox = auto 

[Mask]
External = None 
Auto = False 
SigTh = 10 
FluxImageType = ModelConv 

[Noise]
MinStats = [60, 2] 
BrutalHMP = True 

[HMP]
Alpha = [-1.0, 1.0, 11] 
Scales = [0] 
Ratios = [''] 
NTheta = 6 
SolverMode = PI 
AllowResidIncrease = 0.1 
MajorStallThreshold = 0.8 
Taper = 0 
Support = 0 
PeakWeightImage = None 
Kappa = 0.0 
OuterSpaceTh = 2.0 
FractionRandomPeak = None 

[Hogbom]
PolyFitOrder = 3 
MaxLengthScale = 5 
FreqMode = Poly 
NumBasisFuncs = 12 

[Montblanc]
TensorflowServerTarget =  

[SSDClean]
Parallel = True 
IslandDeconvMode = GA 
SSDSolvePars = ['S', 'Alpha'] 
SSDCostFunc = ['Chi2', 'MinFlux'] 
BICFactor = 0.0 
ArtifactRobust = False 
ConvFFTSwitch = 1000 
NEnlargePars = 0 
NEnlargeData = 2 
RestoreMetroSwitch = 0 
MinMaxGroupDistance = [10, 50] 

[GAClean]
NSourceKin = 50 
NMaxGen = 50 
MinSizeInit = 10 
InitType = HMP 
AlphaInitHMP = [-4.0, 1.0, 6] 
ScalesInitHMP = [0, 1, 2, 4, 8, 16, 24, 32] 
GainInitHMP = 0.1 
RatiosInitHMP = [''] 
NThetaInitHMP = 4 
MaxMinorIterInitHMP = 10000 
AllowNegativeInitHMP = False 
RMSFactorInitHMP = 3.0 
ParallelInitHMP = True 
NCPU = 0 

[MORESANE]
NMajorIter = 200 
NMinorIter = 200 
Gain = 0.1 
ForcePositive = True 
SigmaCutLevel = 1 

[MUFFIN]
mu_s = 0.1 
mu_l = 0.2 
nb = ['(8', '0)'] 
NMinorIter = 200 

[Log]
Memory = False 
Boring = False 
Append = False 

[Debug]
PauseWorkers = False 
FacetPhaseShift = [0.0, 0.0] 
PrintMinorCycleRMS = False 
DumpCleanSolutions = 0 
DumpCleanPostageStamps =  
CleanStallThreshold = 0.0 
MemoryGreedy = True 
APPVerbose = 0 
Pdb = auto 

[Misc]
RandomSeed = None 
ParsetVersion = 0.2 
ConserveMemory = False 

As we can see, quite a lot of options...here are the ones you would likely be concerned with. Note that the general command line syntax for an option in the pattern
[Category]
Option = True

is --Category-Option True   (the = sign is facultative)

In [ ]:
[Data]
MS = DATA/L242400_SB095_uv.dppp.4h-4.5h.MS # name of your ms
ColName = CORRECTED_DATA                   # name of column to image
ChunkHours = 0.0                           # in case of memory error, set to 4, 2, 1...

[Selection]
FlagAnts =                                 # regular expression or strings for ants to flag
UVRangeKm = [0, 2000]                      # self evident

[Output]
Mode = Clean                               # can also be Dirty or PSF
Name = image                               # name of image you're making
RestoringBeam = None                       # set to 4 times cell size by default
Cubes =                                    # if you want freq. cubes. See -h
Images = DdPAMRIikz                        # see DDF.py -h | grep Images 

[Image]
NPix = 500                                 # number of pixels a side
Cell = 5.0                                 # pixel size in arcsec
PhaseCenterRADEC = align                   # where to put centre of image. If align, same as MS phase centre

[Weight]
ColName = WEIGHT_SPECTRUM                  # weight column to use
Mode = Briggs                              # self evident but maybe you want to use uniform or something
Robust = 0.0                               # see above

[RIME]
DecorrMode =                               # set to FT for wide fields of view. it's RIME magic.

[Parallel]
NCPU = 4                                   # number of CPUs used for parallelisation

[Cache]
Reset = True                               # good practice to set this to True when not changing image name.

[Beam]
Model = None                               # can be LOFAR
LOFARBeamMode = AE                         # can be A or AE

[Freq]
NBand = 1                                  # use this when you want to make cubes: get as many freq. slices as NBand

# after that, it's just clean options.

[Deconv]
Mode = HMP 
MaxMajorIter = 20 
MaxMinorIter = 20000 
AllowNegative = True 
Gain = 0.1 
FluxThreshold = 0.0 
CycleFactor = 0.0 
RMSFactor = 0.0 
PeakFactor = 0.15 
PrevPeakFactor = 0.0 
NumRMSSamples = 10000 
ApproximatePSF = 0 
PSFBox = auto 

[Mask]
External = None 
Auto = False 
SigTh = 10 
FluxImageType = ModelConv 


[HMP]
Alpha = [-1.0, 1.0, 11] 
Scales = [0] 
Ratios = [''] 
NTheta = 6 
SolverMode = PI 
AllowResidIncrease = 0.1 
MajorStallThreshold = 0.8 
Taper = 0 
Support = 0 
PeakWeightImage = None 
Kappa = 0.0 
OuterSpaceTh = 2.0 
FractionRandomPeak = None 